In [1]:
import pandas as pd
import folium
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point
from geopandas import GeoDataFrame
import geopandas as gpd
from matplotlib import pyplot as plt
from shapely import wkt
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

In [2]:
import json
import time
import csv
from tqdm import tqdm
from pathlib import Path
import requests

In [3]:
def standartize_address(street):
    
    shorthands = {'ул.': 'улица', 
                    'пр.': 'проспект',
                    'ш.': 'шоссе',
                    'пер.': 'переулок',
                    'бульв.': 'бульвар',
                    'наб.': 'набережная'}
    
    locality = None
    street = street.split(',')
    if len(street) == 2:
        locality = street[0]
        street = street[1]
    else:
        street = street[0]
    street = street.split(' ')
    if len(street) <= 1:
        return(street[0])
    else:
        status = street[-1]
        if status.lower() in shorthands:
            status = shorthands[status.lower()]
        if street[-2][-2:] in 'ая ий ый ое':
            if locality is not None:
                return ' '.join((locality + ',', ' '.join(street[:-1]), status))
            else:
                return ' '.join((' '.join(street[:-1]), status))
        else:
            if locality is not None:
                return ' '.join((locality + ',', status, ' '.join(street[0:-2]), street[-2]))
            else:
                return ' '.join((status, ' '.join(street[0:-2]), street[-2]))

In [4]:
def full_address(locality, number, building):
    if building == 'nan':
        return 'Санкт-Петербург, ' + locality + ' ' + number
    else:
        return 'Санкт-Петербург, ' + locality + ' ' + number + ' к' + building

In [5]:
def geocode(df):
    """
        Query nominatim api for address coordinates.
        Df should have a column named 'addr' formated
        so that nominatim understands it.
        Saves json files.
    """
    url = 'https://nominatim.openstreetmap.org/search'
    for index, row in df.iterrows():
        time.sleep(1)
        try:
            print(index, row['addr'])
            params = {'q': row['addr'], 'format':'json'}
            g = requests.get(url, params=params)
            g = g.json()
            print(g)
            with open(Path('./jsons', str(index), 'w')) as f:
                json.dump(g, f)
        except Exception as err:
            print(err)

In [6]:
def extract_coordinates(df):
    """
        Get coordinates from saved api responses.
        District name from the dataset must be present in the address
        returned by the api. It guarantees that streets with identical names
        are not confused.
    """
    df_addr = pd.DataFrame(columns=['index', 'address', 'class', 'lat', 'lon'])
    for index, row in error.iterrows():
        with open(f'./jsons/{index}', 'r') as f:
            response = json.load(f)
            for item in response:
                candidate = None
                if (row['addr_district'] in item['display_name']):
                    if not candidate:
                        candidate = {'index': index, 'address': item['display_name'],
                                     'class': item['class'], 'lat': item['lat'], 'lon': item['lon']}
                    if (item['class'] == 'building'):
                        candidate = {'index': index, 'address': item['display_name'],
                                     'class': item['class'], 'lat': item['lat'], 'lon': item['lon']}
                        break
            if candidate:
                df_addr = df_addr.append(candidate, ignore_index=True)
    return df_addr

In [10]:
# отделим точки с ошибочными координатами: район, куда точка "упала" и район из датасета не совпадают
districts = pd.read_csv('shapes/districts.csv', index_col=0)
districts['geometry'] = districts['geometry'].apply(wkt.loads)
districts = GeoDataFrame(districts, crs={'init': 'epsg:4326'}, geometry='geometry')

points = pd.read_csv('trash/points.csv', index_col=0)
data = pd.read_csv('data/housing_data.csv')
data = data.join(points, how='inner')
data['geometry'] = data.apply(lambda x: Point(x.lng, x.lat), axis=1)
data = GeoDataFrame(data, crs={'init': 'epsg:4326'}, geometry='geometry')
data = data.to_crs(districts.crs)

full = gpd.sjoin(districts, data, op='contains')
full['name'] = full['name'].str.split(' ')
full['name'] = full['name'].apply(lambda x: x[0])

/home/gustav/Projects/StPHousing/env/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [11]:
error = full[full['name'] != full['addr_district']]
error = error[['index_right', 'addr_street', 'addr_number', 'addr_building', 'addr_letter', 'addr_district']]
error['locality'] = error['addr_street'].map(standartize_address)
error['addr'] = error.apply(lambda x: full_address(x.locality, x.addr_number, str(x.addr_building)), axis=1)
error = error.set_index('index_right')
error.shape

(848, 7)

In [26]:
geocode(error)

20182 Санкт-Петербург, улица  Гоголя 8
[{'place_id': 277552667, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 107956042, 'boundingbox': ['59.7963965', '59.7964979', '30.5162784', '30.5164842'], 'lat': '59.7964472', 'lon': '30.5163812877794', 'display_name': '8, улица Гоголя, Петро-Славянка, Колпинский район, Санкт-Петербург, Северо-Западный федеральный округ, 196642, Россия', 'class': 'building', 'type': 'yes', 'importance': 0.511}, {'place_id': 88963428, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 52247674, 'boundingbox': ['59.8522106', '59.8524272', '30.0254366', '30.0261988'], 'lat': '59.85231885', 'lon': '30.025817698528', 'display_name': '8, улица Гоголя, Нойдорф, Стрельна, Петродворцовый район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия', 'class': 'building', 'type': 'yes', 'importance': 0.511}]
5635 Санкт-Петербург, ули

KeyboardInterrupt: 

In [12]:
results = extract_coordinates(error)

In [13]:
results.groupby(by='class').count()

,index,address,lat,lon
class,,,,
amenity,14,14,14,14
building,471,471,471,471
highway,64,64,64,64
historic,1,1,1,1
man_made,1,1,1,1
office,1,1,1,1
shop,6,6,6,6
tourism,1,1,1,1


In [15]:
full.shape

(19441, 52)

In [14]:
results.shape

(559, 5)

In [9]:
for row in error.iterrows():
    print(row[1][0], row[1][5])
    with open(f'./jsons/{row[0]}', 'r') as f:
        o = json.load(f)
        for i in o:
            if row[1][5] in i['display_name']:
                df_addr = df_addr.append({'number': row[0], 'address': i['display_name'],
                                         'lat': i['lat'], 'lon': i['lon']}, ignore_index=True)
                print(i['display_name'])
                break

М.Садовая ул. М.Садовая улица
Некрасова ул. улица  Некрасова
Софийская ул. Софийская улица
Софийская улица, Колпино, Колпинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Софийская ул. Софийская улица
Софийская улица, Колпино, Колпинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Софийская ул. Софийская улица
Софийская улица, Колпино, Колпинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Губина ул. улица  Губина
Губина ул. улица  Губина
Ленина пр. проспект  Ленина
Гоголя ул. улица  Гоголя
Гоголя ул. улица  Гоголя
Садовая ул. Садовая улица
Садовая улица, Петро-Славянка, Колпинский район, Санкт-Петербург, Северо-Западный федеральный округ, 196642, Россия
Витебская ул. Витебская улица
Витебская улица, Петро-Славянка, Колпинский район, Санкт-Петербург, Северо-Западный федеральный округ, 196642, Россия
Витебская ул. Витебская улица
3, Витебская улица, Петро-Славянка, Колпинский район, Санкт-Петербур

9, 3-я Советская улица, Новая Деревня, Пушкин, Пушкинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
6-я Советская ул. 6-я Советская улица
5, 6-я Советская улица, Новая Деревня, Пушкин, Пушкинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
2-я Советская ул. 2-я Советская улица
12, 2-я Советская улица, Новая Деревня, Пушкин, Пушкинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
5-я Советская ул. 5-я Советская улица
10, 5-я Советская улица, Новая Деревня, Пушкин, Пушкинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
3-я Советская ул. 3-я Советская улица
10, 3-я Советская улица, Новая Деревня, Пушкин, Пушкинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
4-я Советская ул. 4-я Советская улица
10, 4-я Советская улица, Новая Деревня, Пушкин, Пушкинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
4-я Советская 

24, Социалистическая улица, Зверинец, Павловск, Пушкинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Правды ул. улица  Правды
Правды ул. улица  Правды
Декабристов ул. улица  Декабристов
Декабристов ул. улица  Декабристов
Красного Курсанта ул. улица Красного Курсанта
25, улица Красного Курсанта, Зверинец, Павловск, Пушкинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Социалистическая ул. Социалистическая улица
8, Социалистическая улица, Зверинец, Павловск, Пушкинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Гоголя ул. улица  Гоголя
Декабристов ул. улица  Декабристов
Социалистическая ул. Социалистическая улица
6, Социалистическая улица, Зверинец, Павловск, Пушкинский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Декабристов ул. улица  Декабристов
Гоголя ул. улица  Гоголя
Красного Курсанта ул. улица Красного Курсанта
34, улица Красного Курсанта, Зверинец, Павловск, 

12, Боровая улица, Фабричный посёлок, Скачки, Красное Село, Красносельский район, Санкт-Петербург, Северо-Западный федеральный округ, 198320, Россия
Боровая ул. Боровая улица
12, Боровая улица, Фабричный посёлок, Скачки, Красное Село, Красносельский район, Санкт-Петербург, Северо-Западный федеральный округ, 198320, Россия
Боровая ул. Боровая улица
22, Боровая улица, Фабричный посёлок, Скачки, Красное Село, Красносельский район, Санкт-Петербург, Северо-Западный федеральный округ, 198320, Россия
Боровая ул. Боровая улица
3, Боровая улица, Фабричный посёлок, Скачки, Красное Село, Красносельский район, Санкт-Петербург, Северо-Западный федеральный округ, 198320, Россия
Боровая ул. Боровая улица
20, Боровая улица, Фабричный посёлок, Скачки, Красное Село, Красносельский район, Санкт-Петербург, Северо-Западный федеральный округ, 198320, Россия
Боровая ул. Боровая улица
Боровая улица, Фабричный посёлок, Скачки, Красное Село, Красносельский район, Санкт-Петербург, Северо-Западный федеральный окр

22, Ивановская улица, Мартышкино, Ломоносов, Петродворцовый район, Санкт-Петербург, Северо-Западный федеральный округ, 198411, Россия
Ивановская ул. Ивановская улица
29, Ивановская улица, Мартышкино, Ломоносов, Петродворцовый район, Санкт-Петербург, Северо-Западный федеральный округ, 198411, Россия
Ивановская ул. Ивановская улица
20, Ивановская улица, Мартышкино, Ломоносов, Петродворцовый район, Санкт-Петербург, Северо-Западный федеральный округ, 198411, Россия
Ивановская ул. Ивановская улица
18, Ивановская улица, Мартышкино, Ломоносов, Петродворцовый район, Санкт-Петербург, Северо-Западный федеральный округ, 198411, Россия
Красная ул. Красная улица
22, Красная улица, Мартышкино, Ломоносов, Петродворцовый район, Санкт-Петербург, Северо-Западный федеральный округ, 198411, Россия
Ивановская ул. Ивановская улица
14, Ивановская улица, Мартышкино, Ломоносов, Петродворцовый район, Санкт-Петербург, Северо-Западный федеральный округ, 198411, Россия
Ивановская ул. Ивановская улица
12, Ивановска

Апраксин пер. переулок  Апраксин
Гражданская ул. Гражданская улица
25, Гражданская улица, Сенной округ, Адмиралтейский район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Апраксин пер. переулок  Апраксин
Фонтанки реки наб. набережная Фонтанки реки
Фонтанки реки наб. набережная Фонтанки реки
Фонтанки реки наб. набережная Фонтанки реки
Апраксин пер. переулок  Апраксин
Апраксин пер. переулок  Апраксин
3-я 1-й половины линия линия 3-я 1-й половины
3-я 1-й половины линия линия 3-я 1-й половины
3-я 1-й половины линия линия 3-я 1-й половины
г.Сестрорецк, 2-я линия г.Сестрорецк, линия  2-я
3-я 1-й половины линия линия 3-я 1-й половины
3-я 1-й половины линия линия 3-я 1-й половины
3-я 1-й половины линия линия 3-я 1-й половины
3-я 1-й половины линия линия 3-я 1-й половины
г. Сестрорецк, 10-я линия г. Сестрорецк, линия  10-я
г.Сестрорецк, 3-я линия г.Сестрорецк, линия  3-я
г.Сестрорецк, 3-я линия г.Сестрорецк, линия  3-я
г.Сестрорецк, 2-я линия г.Сестрорецк, линия  2-я
г.Сес

улица Всеволода Вишневского, Кронштадт, Кронштадтский район, Санкт-Петербург, Северо-Западный федеральный округ, 197760, Россия
Всеволода Вишневского ул. улица Всеволода Вишневского
18, улица Всеволода Вишневского, Кронштадт, Кронштадтский район, Санкт-Петербург, Северо-Западный федеральный округ, 197760, Россия
Гражданская ул. Гражданская улица
Гражданская улица, Кронштадт, Кронштадтский район, Санкт-Петербург, Северо-Западный федеральный округ, 197760, Россия
Гражданская ул. Гражданская улица
8, Гражданская улица, Кронштадт, Кронштадтский район, Санкт-Петербург, Северо-Западный федеральный округ, 197760, Россия
Гражданская ул. Гражданская улица
10, Гражданская улица, Кронштадт, Кронштадтский район, Санкт-Петербург, Северо-Западный федеральный округ, 197760, Россия
Гражданская ул. Гражданская улица
Шаверма, 9, Гражданская улица, Кронштадт, Кронштадтский район, Санкт-Петербург, Северо-Западный федеральный округ, 197760, Россия
Всеволода Вишневского ул. улица Всеволода Вишневского
13, у

8, Кузнечный переулок, Зеленогорск, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Средний пр. Средний проспект
18, Средний проспект, Зеленогорск, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Кузнечный пер. Кузнечный переулок
Кузнечный переулок, Зеленогорск, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Кузнечный пер. Кузнечный переулок
6, Кузнечный переулок, Зеленогорск, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Средний пр. Средний проспект
17, Средний проспект, Зеленогорск, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Мира ул. улица  Мира
Средний пр. Средний проспект
Средний проспект, Зеленогорск, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 190000, Россия
Средний пр. Средний проспект
Средний проспект, Зеленогорск, Курортный район, Санкт-Петербург, Северо-Западный феде

6, Транспортный переулок, Разлив, Сестрорецк, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 197706, Россия
Транспортный пер. Транспортный переулок
6, Транспортный переулок, Разлив, Сестрорецк, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 197706, Россия
Гагаринская ул. Гагаринская улица
34, Гагаринская улица, Тарховка, Сестрорецк, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 197701, Россия
Гагаринская ул. Гагаринская улица
32, Гагаринская улица, Тарховка, Сестрорецк, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 197701, Россия
Гагаринская ул. Гагаринская улица
30, Гагаринская улица, Тарховка, Сестрорецк, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 197701, Россия
Гагаринская ул. Гагаринская улица
25, Гагаринская улица, Тарховка, Сестрорецк, Курортный район, Санкт-Петербург, Северо-Западный федеральный округ, 197701, Россия
Гагаринская ул. Гагаринская улица
28, Гагаринска

Сердобольская улица, Выборгская сторона, округ Чёрная речка, Приморский район, Санкт-Петербург, Северо-Западный федеральный округ, 197342, Россия
Берёзовая аллея Берёзовая аллея
Берёзовая аллея, Ольгино, округ Лахта-Ольгино, Приморский район, Санкт-Петербург, Северо-Западный федеральный округ, 197229, Россия
Берёзовая аллея Берёзовая аллея
Берёзовая аллея, Ольгино, округ Лахта-Ольгино, Приморский район, Санкт-Петербург, Северо-Западный федеральный округ, 197229, Россия
Вокзальная ул. Вокзальная улица
4, Вокзальная улица, Ольгино, округ Лахта-Ольгино, Приморский район, Санкт-Петербург, Северо-Западный федеральный округ, 197229, Россия
Вокзальная ул. Вокзальная улица
2, Вокзальная улица, Ольгино, округ Лахта-Ольгино, Приморский район, Санкт-Петербург, Северо-Западный федеральный округ, 197229, Россия
Советская ул. Советская улица
31, Советская улица, Ольгино, округ Лахта-Ольгино, Приморский район, Санкт-Петербург, Северо-Западный федеральный округ, 197229, Россия
Советская ул. Советская 

In [10]:
df_addr = df_addr.set_index('number')
error = error.join(df_addr)

In [11]:
error.head()

,addr_street,addr_number,addr_building,addr_letter,addr_district,locality,addr,address,lat,lon
index_right,,,,,,,,,,
5263,М.Садовая ул.,3/54,NaN,А,Центральный,М.Садовая улица,"Санкт-Петербург, М.Садовая улица 3/54",NaN,NaN,NaN
5038,Некрасова ул.,45,NaN,А,Центральный,улица Некрасова,"Санкт-Петербург, улица Некрасова 45",NaN,NaN,NaN
13237,Софийская ул.,3,2,А,Выборгский,Софийская улица,"Санкт-Петербург, Софийская улица 3 к2","Софийская улица, Колпино, Колпинский район, Са...",59.7491085,30.5301987
1137,Софийская ул.,32,2,А,Фрунзенский,Софийская улица,"Санкт-Петербург, Софийская улица 32 к2","Софийская улица, Колпино, Колпинский район, Са...",59.7491085,30.5301987
18415,Софийская ул.,26-28,NaN,А,Выборгский,Софийская улица,"Санкт-Петербург, Софийская улица 26-28","Софийская улица, Колпино, Колпинский район, Са...",59.7491085,30.5301987


In [19]:
results = results.set_index('index')
points_correction = results[['lat', 'lon']]
points_correction.columns = ['lat', 'lng']

In [20]:
points_correction

,lat,lng
index,,
5038,59.9390205,30.3532303
1137,59.8729074,30.4047006
1267,59.8950031,30.263117
10569,59.8950031,30.263117
20182,59.85231885,30.025817698528
...,...,...
16794,59.9934289,29.778574
7388,59.9934289,29.778574
18278,59.92500335,30.3150670187211


In [32]:
points = pd.read_csv('trash/points.csv', index_col=0)
points = points.drop(error.index)

In [33]:
points.shape

(18567, 2)

In [34]:
points = points.append(points_correction)

In [35]:
points.to_csv('points_corr.csv')

In [36]:
points.shape

(19126, 2)